In [ ]:
!pip uninstall -y transformers huggingface_hub accelerate
!pip install -U transformers==4.45.0 huggingface_hub==0.25.2 datasets==3.0.1 tensorflow==2.17.0

!pip install -U transformers datasets huggingface_hub tensorflow

In [ ]:
!pip install --upgrade --force-reinstall transformers==4.44.2 huggingface_hub==0.24.5 datasets tensorflow

## New Start

In [ ]:
!pip install --force-reinstall "numpy<2" "matplotlib<3.9" "scikit-learn<1.5" tensorflow transformers==4.44.2 huggingface_hub==0.24.5 datasets

In [ ]:
def create_sentence_pairs(sentences, tokenizer, max_len=128):
    pairs, labels = [], []
    for i in range(len(sentences) - 1):
        # 50% positive (consecutive)
        if random.random() > 0.5:
            first, second = sentences[i], sentences[i+1]
            label = 1
        # 50% negative (random)
        else:
            first = sentences[i]
            second = random.choice(sentences)
            label = 0
        encoded = tokenizer(
            first, second,
            max_length=max_len,
            truncation=True,
            padding="max_length",
            return_tensors="tf"
        )
        pairs.append(encoded)
        labels.append(label)
    return pairs, labels

train_pairs, train_labels = create_sentence_pairs(train_texts, tokenizer)
valid_pairs, valid_labels = create_sentence_pairs(valid_texts, tokenizer)
print(f"Train pairs: {len(train_pairs)}, Validation pairs: {len(valid_pairs)}")

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
def mask_tokens(inputs, tokenizer, mlm_probability=0.15):
    inputs = tf.identity(inputs)
    labels = tf.identity(inputs)
    probability_matrix = tf.random.uniform(shape=inputs.shape)
    mask_arr = probability_matrix < mlm_probability

    # Don't mask special tokens
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True)
        for val in inputs.numpy().tolist()
    ]
    mask_arr = mask_arr & ~tf.constant(special_tokens_mask, dtype=tf.bool)

    labels = tf.where(mask_arr, labels, -100)  # only compute loss on masked tokens

    # 80% MASK, 10% random, 10% unchanged
    mask_indices = tf.where(mask_arr)
    random_indices = tf.random.uniform(mask_arr.shape) < 0.1
    random_words = tf.random.uniform(inputs.shape, maxval=tokenizer.vocab_size, dtype=tf.int32)

    # Apply masking
    inputs = tf.where(mask_arr & ~random_indices, tokenizer.mask_token_id, inputs)
    inputs = tf.where(mask_arr & random_indices, random_words, inputs)
    return inputs, labels

In [ ]:
def mask_tokens(inputs, tokenizer, mlm_probability=0.15):
    """Prepare masked tokens inputs/labels for masked language modeling."""
    inputs = tf.convert_to_tensor(inputs)
    labels = tf.identity(inputs)

    # Create mask for non-special tokens
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True)
        for val in inputs.numpy().tolist()
    ]
    # Convert to flat boolean tensor
    special_tokens_mask = np.array(special_tokens_mask).astype(bool)

    # Random mask for MLM
    probability_matrix = np.random.rand(*inputs.shape)
    mask_arr = probability_matrix < mlm_probability

    # Remove masking of special tokens
    mask_arr = mask_arr & ~special_tokens_mask

    # Masked labels
    labels = tf.where(mask_arr, labels, -100)

    # Replace masked input tokens with [MASK] token ID
    mask_token_id = tokenizer.mask_token_id
    inputs = tf.where(mask_arr, mask_token_id, inputs)

    return inputs, labels

In [ ]:
model = TFBertForPreTraining.from_pretrained("bert-base-uncased")
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

All PyTorch model weights were used when initializing TFBertForPreTraining.

All the weights of TFBertForPreTraining were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForPreTraining for predictions without further training.


In [ ]:
def make_dataset(pairs, nsp_labels, batch_size=8, max_len=128):
    input_ids, attention_masks, token_type_ids, mlm_labels, nsp_labels_out = [], [], [], [], []

    for i, encoded in enumerate(pairs):
        ids, labels = mask_tokens(encoded["input_ids"], tokenizer)
        input_ids.append(ids)
        attention_masks.append(encoded["attention_mask"])
        token_type_ids.append(encoded["token_type_ids"])
        mlm_labels.append(labels)
        nsp_labels_out.append(nsp_labels[i])

    def pad(x, length=max_len):
        x = tf.convert_to_tensor(x)
        x = tf.squeeze(x)  # remove extra dimensions like [1, N]
        pad_len = length - tf.shape(x)[0]
        pad_len = tf.maximum(pad_len, 0)
        return tf.pad(x, [[0, pad_len]])[:length]  # crop if too long

    input_ids = tf.stack([pad(x) for x in input_ids])
    attention_masks = tf.stack([pad(x) for x in attention_masks])
    token_type_ids = tf.stack([pad(x) for x in token_type_ids])
    mlm_labels = tf.stack([pad(x) for x in mlm_labels])

    ds = tf.data.Dataset.from_tensor_slices((
        {
            "input_ids": input_ids,
            "attention_mask": attention_masks,
            "token_type_ids": token_type_ids,
        },
        {
            "labels": mlm_labels,
            "next_sentence_label": tf.constant(nsp_labels_out, dtype=tf.int32),
        }
    ))

    return ds.shuffle(1000).batch(batch_size)

In [ ]:
train_ds = make_dataset(train_pairs[:5000], train_labels[:5000])
valid_ds = make_dataset(valid_pairs[:1000], valid_labels[:1000])

In [ ]:
epochs = 2

for epoch in range(epochs):
    print(f"\n🌟 Epoch {epoch+1}/{epochs} 🌟")
    for batch, (inputs, labels) in enumerate(train_ds):
        with tf.GradientTape() as tape:
            outputs = model(**inputs, labels=labels["labels"], next_sentence_label=labels["next_sentence_label"], training=True)
            loss = outputs.loss
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if batch % 50 == 0:
            print(f"Batch {batch}: Loss = {loss.numpy().item():.4f}")


🌟 Epoch 1/2 🌟
Batch 0: Loss = 9.4518
Batch 50: Loss = 2.5449
Batch 100: Loss = 3.1371
Batch 150: Loss = 2.9186
Batch 200: Loss = 1.8927
Batch 250: Loss = 2.6230
Batch 300: Loss = 2.4102
Batch 350: Loss = 1.8844
Batch 400: Loss = 2.1412
Batch 450: Loss = 2.5805
Batch 500: Loss = 2.6488
Batch 550: Loss = 1.8577
Batch 600: Loss = 1.9257

🌟 Epoch 2/2 🌟
Batch 0: Loss = 1.6188
Batch 50: Loss = 1.2366
Batch 100: Loss = 1.5507
Batch 150: Loss = 1.9639
Batch 200: Loss = 1.5223
Batch 250: Loss = 1.6239
Batch 300: Loss = 1.8290
Batch 350: Loss = 1.2868
Batch 400: Loss = 1.6175
Batch 450: Loss = 1.5182
Batch 500: Loss = 1.6605
Batch 550: Loss = 1.8960
Batch 600: Loss = 1.0730


In [ ]:
val_loss_total = 0
count = 0
for inputs, labels in valid_ds:
    outputs = model(**inputs, labels=labels["labels"], next_sentence_label=labels["next_sentence_label"], training=False)
    val_loss_total += outputs.loss.numpy()
    count += 1

print(f"\n Validation Loss: {val_loss_total / count:.4f}")